In [1]:
import gc
import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools

In [2]:
def get_col_types(train):
    cat_features = [
            "B_30",
            "B_38",
            "D_114",
            "D_116",
            "D_117",
            "D_120",
            "D_126",
            "D_63",
            "D_64",
            "D_66",
            "D_68",
        ]
    num_features = [col for col in train.columns.to_list() 
                    if col not in cat_features + ["customer_ID", "S_2"]]
    return cat_features, num_features

def get_num_agg(train, num_features):
    train_num_agg = train.groupby("customer_ID")[num_features].agg(['last', 'mean', 'std', 'min', 'max'])
    train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
    train_num_agg.reset_index(inplace = True)
    
    cols = list(train_num_agg.dtypes[train_num_agg.dtypes == 'float64'].index)
    train_num_agg[cols] = train_num_agg[cols].astype(np.float32)
    return train_num_agg

def get_cat_agg(train, cat_features):
    train_cat_agg = train.groupby("customer_ID")[cat_features].agg(['last'])
    train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
    train_cat_agg.reset_index(inplace = True)
    train_cat_agg

    cols = list(train_cat_agg.dtypes[train_cat_agg.dtypes == 'int64'].index)
    train_cat_agg[cols] = train_cat_agg[cols].astype(np.int32)
    return train_cat_agg

def get_difference(data, num_features):
    df1 = []
    customer_ids = []
    for customer_id, df in tqdm(data.groupby(['customer_ID'])):
        diff_df1 = df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        df1.append(diff_df1)
        customer_ids.append(customer_id)
    df1 = np.concatenate(df1, axis = 0)
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
    df1['customer_ID'] = customer_ids
    return df1

def preprocessing(train, is_test: bool = False):
    cat_features, num_features = get_col_types(train)
    
    train_num_agg = get_num_agg(train, num_features)
    train_cat_agg = get_cat_agg(train, cat_features)
    train_diff = get_difference(train, num_features)

    train_agg = (
        train_num_agg
            .merge(train_cat_agg, how = 'inner', on = 'customer_ID')
            .merge(train_diff, how = 'inner', on = 'customer_ID')
    )
    
    if not is_test:
        train_labels = pd.read_csv("../input/train_labels.csv")
        train_agg = train_agg.merge(train_labels, how = 'inner', on = 'customer_ID')
    
    del train_num_agg, train_cat_agg, train_diff
    gc.collect()
    return train_agg

In [3]:
train = pd.read_parquet("../input/train_data.parquet")
train_agg = preprocessing(train, is_test=False)
train_agg

  0%|          | 0/458913 [00:00<?, ?it/s]

,customer_ID,P_2_last,P_2_mean,P_2_std,P_2_min,P_2_max,D_39_last,D_39_mean,D_39_std,D_39_min,D_39_max,B_1_last,B_1_mean,B_1_std,B_1_min,B_1_max,B_2_last,B_2_mean,B_2_std,B_2_min,B_2_max,R_1_last,R_1_mean,R_1_std,R_1_min,R_1_max,S_3_last,S_3_mean,S_3_std,S_3_min,S_3_max,D_41_last,D_41_mean,D_41_std,D_41_min,D_41_max,B_3_last,B_3_mean,B_3_std,B_3_min,B_3_max,D_42_last,D_42_mean,D_42_std,D_42_min,D_42_max,D_43_last,D_43_mean,D_43_std,D_43_min,D_43_max,D_44_last,D_44_mean,D_44_std,D_44_min,D_44_max,B_4_last,B_4_mean,B_4_std,B_4_min,B_4_max,D_45_last,D_45_mean,D_45_std,D_45_min,D_45_max,B_5_last,B_5_mean,B_5_std,B_5_min,B_5_max,R_2_last,R_2_mean,R_2_std,R_2_min,R_2_max,D_46_last,D_46_mean,D_46_std,D_46_min,D_46_max,D_47_last,D_47_mean,D_47_std,D_47_min,D_47_max,D_48_last,D_48_mean,D_48_std,D_48_min,D_48_max,D_49_last,D_49_mean,D_49_std,D_49_min,D_49_max,B_6_last,B_6_mean,B_6_std,B_6_min,B_6_max,B_7_last,B_7_mean,B_7_std,B_7_min,B_7_max,B_8_last,B_8_mean,B_8_std,B_8_min,B_8_max,D_50_last,D_50_mean,D_50_std,D_50_min,D_50_max,D_51_last,D_51_mean,D_51_std,D_51_min,D_51_max,B_9_last,B_9_mean,B_9_std,B_9_min,B_9_max,R_3_last,R_3_mean,R_3_std,R_3_min,R_3_max,D_52_last,D_52_mean,D_52_std,D_52_min,D_52_max,P_3_last,P_3_mean,P_3_std,P_3_min,P_3_max,B_10_last,B_10_mean,B_10_std,B_10_min,B_10_max,D_53_last,D_53_mean,D_53_std,D_53_min,D_53_max,S_5_last,S_5_mean,S_5_std,S_5_min,S_5_max,B_11_last,B_11_mean,B_11_std,B_11_min,B_11_max,S_6_last,S_6_mean,S_6_std,S_6_min,S_6_max,D_54_last,D_54_mean,D_54_std,D_54_min,D_54_max,R_4_last,R_4_mean,R_4_std,R_4_min,R_4_max,S_7_last,S_7_mean,S_7_std,S_7_min,S_7_max,B_12_last,B_12_mean,B_12_std,B_12_min,B_12_max,S_8_last,S_8_mean,S_8_std,S_8_min,S_8_max,D_55_last,D_55_mean,D_55_std,D_55_min,D_55_max,D_56_last,D_56_mean,D_56_std,D_56_min,D_56_max,B_13_last,B_13_mean,B_13_std,B_13_min,B_13_max,R_5_last,R_5_mean,R_5_std,R_5_min,R_5_max,D_58_last,D_58_mean,D_58_std,D_58_min,D_58_max,S_9_last,S_9_mean,S_9_std,S_9_min,S_9_max,B_14_last,B_14_mean,B_14_std,B_14_min,B_14_max,D_59_last,D_59_mean,D_59_std,D_59_min,D_59_max,D_60_last,D_60_mean,D_60_std,D_60_min,D_60_max,D_61_last,D_61_mean,D_61_std,D_61_min,D_61_max,B_15_last,B_15_mean,B_15_std,B_15_min,B_15_max,S_11_last,S_11_mean,S_11_std,S_11_min,...,R_28_max,D_134_last,D_134_mean,D_134_std,D_134_min,D_134_max,D_135_last,D_135_mean,D_135_std,D_135_min,D_135_max,D_136_last,D_136_mean,D_136_std,D_136_min,D_136_max,D_137_last,D_137_mean,D_137_std,D_137_min,D_137_max,D_138_last,D_138_mean,D_138_std,D_138_min,D_138_max,D_139_last,D_139_mean,D_139_std,D_139_min,D_139_max,D_140_last,D_140_mean,D_140_std,D_140_min,D_140_max,D_141_last,D_141_mean,D_141_std,D_141_min,D_141_max,D_142_last,D_142_mean,D_142_std,D_142_min,D_142_max,D_143_last,D_143_mean,D_143_std,D_143_min,D_143_max,D_144_last,D_144_mean,D_144_std,D_144_min,D_144_max,D_145_last,D_145_mean,D_145_std,D_145_min,D_145_max,B_30_last,B_38_last,D_114_last,D_116_last,D_117_last,D_120_last,D_126_last,D_63_last,D_64_last,D_66_last,D_68_last,P_2_diff1,D_39_diff1,B_1_diff1,B_2_diff1,R_1_diff1,S_3_diff1,D_41_diff1,B_3_diff1,D_42_diff1,D_43_diff1,D_44_diff1,B_4_diff1,D_45_diff1,B_5_diff1,R_2_diff1,D_46_diff1,D_47_diff1,D_48_diff1,D_49_diff1,B_6_diff1,B_7_diff1,B_8_diff1,D_50_diff1,D_51_diff1,B_9_diff1,R_3_diff1,D_52_diff1,P_3_diff1,B_10_diff1,D_53_diff1,S_5_diff1,B_11_diff1,S_6_diff1,D_54_diff1,R_4_diff1,S_7_diff1,B_12_diff1,S_8_diff1,D_55_diff1,D_56_diff1,B_13_diff1,R_5_diff1,D_58_diff1,S_9_diff1,B_14_diff1,D_59_diff1,D_60_diff1,D_61_diff1,B_15_diff1,S_11_diff1,D_62_diff1,D_65_diff1,B_16_diff1,B_17_diff1,B_18_diff1,B_19_diff1,B_20_diff1,S_12_diff1,R_6_diff1,S_13_diff1,B_21_diff1,D_69_diff1,B_22_diff1,D_70_diff1,D_71_diff1,D_72_diff1,S_15_diff1,B_23_diff1,D_73_diff1,P_4_diff1,D_74_diff1,D_75_diff1,D_76_diff1,B_24_diff1,R_7_diff1,D_77_diff1,B_25_diff1,B_26_diff1,D_78_diff1,D_79_diff1,R_8_diff1,R_9_diff1,S_16_diff1,D_80_diff1,R_10_diff1,R_11_diff1,B_27_diff1,D_81_diff1,D_82_diff1,S_17_diff1,R_12_diff1,B_28_diff1,R_13_diff1,D_83_dif

In [4]:
train_agg.to_parquet("../tmp/train_agg.parquet")

del train, train_agg
gc.collect()

0

In [5]:
test = pd.read_parquet("../input/test_data.parquet")
test_agg = preprocessing(test, is_test=True)
test_agg

  0%|          | 0/924621 [00:00<?, ?it/s]

,customer_ID,P_2_last,P_2_mean,P_2_std,P_2_min,P_2_max,D_39_last,D_39_mean,D_39_std,D_39_min,D_39_max,B_1_last,B_1_mean,B_1_std,B_1_min,B_1_max,B_2_last,B_2_mean,B_2_std,B_2_min,B_2_max,R_1_last,R_1_mean,R_1_std,R_1_min,R_1_max,S_3_last,S_3_mean,S_3_std,S_3_min,S_3_max,D_41_last,D_41_mean,D_41_std,D_41_min,D_41_max,B_3_last,B_3_mean,B_3_std,B_3_min,B_3_max,D_42_last,D_42_mean,D_42_std,D_42_min,D_42_max,D_43_last,D_43_mean,D_43_std,D_43_min,D_43_max,D_44_last,D_44_mean,D_44_std,D_44_min,D_44_max,B_4_last,B_4_mean,B_4_std,B_4_min,B_4_max,D_45_last,D_45_mean,D_45_std,D_45_min,D_45_max,B_5_last,B_5_mean,B_5_std,B_5_min,B_5_max,R_2_last,R_2_mean,R_2_std,R_2_min,R_2_max,D_46_last,D_46_mean,D_46_std,D_46_min,D_46_max,D_47_last,D_47_mean,D_47_std,D_47_min,D_47_max,D_48_last,D_48_mean,D_48_std,D_48_min,D_48_max,D_49_last,D_49_mean,D_49_std,D_49_min,D_49_max,B_6_last,B_6_mean,B_6_std,B_6_min,B_6_max,B_7_last,B_7_mean,B_7_std,B_7_min,B_7_max,B_8_last,B_8_mean,B_8_std,B_8_min,B_8_max,D_50_last,D_50_mean,D_50_std,D_50_min,D_50_max,D_51_last,D_51_mean,D_51_std,D_51_min,D_51_max,B_9_last,B_9_mean,B_9_std,B_9_min,B_9_max,R_3_last,R_3_mean,R_3_std,R_3_min,R_3_max,D_52_last,D_52_mean,D_52_std,D_52_min,D_52_max,P_3_last,P_3_mean,P_3_std,P_3_min,P_3_max,B_10_last,B_10_mean,B_10_std,B_10_min,B_10_max,D_53_last,D_53_mean,D_53_std,D_53_min,D_53_max,S_5_last,S_5_mean,S_5_std,S_5_min,S_5_max,B_11_last,B_11_mean,B_11_std,B_11_min,B_11_max,S_6_last,S_6_mean,S_6_std,S_6_min,S_6_max,D_54_last,D_54_mean,D_54_std,D_54_min,D_54_max,R_4_last,R_4_mean,R_4_std,R_4_min,R_4_max,S_7_last,S_7_mean,S_7_std,S_7_min,S_7_max,B_12_last,B_12_mean,B_12_std,B_12_min,B_12_max,S_8_last,S_8_mean,S_8_std,S_8_min,S_8_max,D_55_last,D_55_mean,D_55_std,D_55_min,D_55_max,D_56_last,D_56_mean,D_56_std,D_56_min,D_56_max,B_13_last,B_13_mean,B_13_std,B_13_min,B_13_max,R_5_last,R_5_mean,R_5_std,R_5_min,R_5_max,D_58_last,D_58_mean,D_58_std,D_58_min,D_58_max,S_9_last,S_9_mean,S_9_std,S_9_min,S_9_max,B_14_last,B_14_mean,B_14_std,B_14_min,B_14_max,D_59_last,D_59_mean,D_59_std,D_59_min,D_59_max,D_60_last,D_60_mean,D_60_std,D_60_min,D_60_max,D_61_last,D_61_mean,D_61_std,D_61_min,D_61_max,B_15_last,B_15_mean,B_15_std,B_15_min,B_15_max,S_11_last,S_11_mean,S_11_std,S_11_min,...,R_28_min,R_28_max,D_134_last,D_134_mean,D_134_std,D_134_min,D_134_max,D_135_last,D_135_mean,D_135_std,D_135_min,D_135_max,D_136_last,D_136_mean,D_136_std,D_136_min,D_136_max,D_137_last,D_137_mean,D_137_std,D_137_min,D_137_max,D_138_last,D_138_mean,D_138_std,D_138_min,D_138_max,D_139_last,D_139_mean,D_139_std,D_139_min,D_139_max,D_140_last,D_140_mean,D_140_std,D_140_min,D_140_max,D_141_last,D_141_mean,D_141_std,D_141_min,D_141_max,D_142_last,D_142_mean,D_142_std,D_142_min,D_142_max,D_143_last,D_143_mean,D_143_std,D_143_min,D_143_max,D_144_last,D_144_mean,D_144_std,D_144_min,D_144_max,D_145_last,D_145_mean,D_145_std,D_145_min,D_145_max,B_30_last,B_38_last,D_114_last,D_116_last,D_117_last,D_120_last,D_126_last,D_63_last,D_64_last,D_66_last,D_68_last,P_2_diff1,D_39_diff1,B_1_diff1,B_2_diff1,R_1_diff1,S_3_diff1,D_41_diff1,B_3_diff1,D_42_diff1,D_43_diff1,D_44_diff1,B_4_diff1,D_45_diff1,B_5_diff1,R_2_diff1,D_46_diff1,D_47_diff1,D_48_diff1,D_49_diff1,B_6_diff1,B_7_diff1,B_8_diff1,D_50_diff1,D_51_diff1,B_9_diff1,R_3_diff1,D_52_diff1,P_3_diff1,B_10_diff1,D_53_diff1,S_5_diff1,B_11_diff1,S_6_diff1,D_54_diff1,R_4_diff1,S_7_diff1,B_12_diff1,S_8_diff1,D_55_diff1,D_56_diff1,B_13_diff1,R_5_diff1,D_58_diff1,S_9_diff1,B_14_diff1,D_59_diff1,D_60_diff1,D_61_diff1,B_15_diff1,S_11_diff1,D_62_diff1,D_65_diff1,B_16_diff1,B_17_diff1,B_18_diff1,B_19_diff1,B_20_diff1,S_12_diff1,R_6_diff1,S_13_diff1,B_21_diff1,D_69_diff1,B_22_diff1,D_70_diff1,D_71_diff1,D_72_diff1,S_15_diff1,B_23_diff1,D_73_diff1,P_4_diff1,D_74_diff1,D_75_diff1,D_76_diff1,B_24_diff1,R_7_diff1,D_77_diff1,B_25_diff1,B_26_diff1,D_78_diff1,D_79_diff1,R_8_diff1,R_9_diff1,S_16_diff1,D_80_diff1,R_10_diff1,R_11_diff1,B_27_diff1,D_81_diff1,D_82_diff1,S_17_diff1,R_12_diff1,B_28_diff1,R_13_diff1

In [6]:
test_agg.to_parquet("../tmp/test_agg.parquet")

del test, test_agg
gc.collect()

0